### Objective

> Tensorflow의 Session에 대해 이해해보자

In [1]:
%matplotlib inline
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important;}</style>"))

## Session이란?

> Graph와 Session은 클래스와 Instance의 관계로 볼 수 있다. Graph는 우리가 어떤 식으로 작업을 할지에 대해 명세를 하는 거라면, Session은 주어진 Graph에 따라, 실제로 계산을 처리하는 것이다. Graph 단계에서는 Class처럼 실제 메모리 공간에 위치하는 것이 아니고, Session을 했을 때에만 비로소 메모리 공간을 할당받기 시작한다. 

텐서플로우의 모든 연산의 구현은 C++와 Cuda로 짜져 있다. 우리가 파이썬에서 진행하는 모든 작업들은 실제로는 어떤 연산을 할지 계획을 하고, 그것을 Graph로 표현하는 행위이다. Tensorflow에서의 Graph들은 GRPC를 통해, Python과 C++를 **통신**하는 형태로 되어 있다. 
실제로 모든 graph들은 `protobuffer` 타입으로 저장되고, 이를 바탕으로 동작된다. 

그렇기 때문에, 나중에 Python 외에도 Android, Swift에서도 쌩쌩 잘 돌아가길 기대할 수 있는 이유 중 하나가 바로 여기에 있다. 지금 개발된 모든 모델들이, model의 protobuffer을 reading할 수 잇는 시스템만 구현되면, 재사용을 할 수 있기 때문이다.



* grpc란?

> 구글에서 만든 remote procedure call. 다른 프로세스 간, 다른 서버 간 메소드들을 호출해주는 것이다. Python 프로세스가 C++ 프로세스를 원격으로 조작하도록 만들어줌. 

![](https://cdn-images-1.medium.com/max/1600/1*-iPT4dK2jSvdzNUXQrKK5w.png)

In [2]:
graph = tf.Graph()

with graph.as_default():
    X = tf.placeholder(tf.float32, (1,3), name='X')    
    W = tf.get_variable("weight", shape=(3,3), 
                        initializer=tf.initializers.random_normal())
    y = tf.matmul(X,W)
    
    init_op = tf.global_variables_initializer()

> 실제로 프로토버퍼 타입의 내용들은 아래와 같이 기록된다.

In [3]:
graph.as_graph_def()

node {
  name: "X"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 1
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "weight/Initializer/random_normal/shape"
  op: "Const"
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@weight"
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\003\000\000\000\003\000\000\000"
      }
    }
  }
}
node {
  name: "weight/Initializer/random_normal/mean"
  op: "Const"
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@weight"
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    va

> 파이썬은 이 내용을 C++ 프로세스에게 전달하고, c++은 이것을 가지고 연산을 한다. 

In [4]:
sess = tf.Session(graph=graph) # graph를 명세해주지 않으면, default_graph로 session을 만듦

sess.run(init_op) # Variable을 초기화 시켜줌
sess.run(y, feed_dict={X:[[1.2,2.1,3.2]]})

array([[-6.3482347 ,  0.43590015,  2.997642  ]], dtype=float32)

# Session을 통해 실제로 돌려보자!

## sess.run(fetches, feed_dict)

* fetches : graph의 element
* feed_dict : fetches에 대응되는 placeholder 리스트들

일전에 말했듯, graph의 element는 크게 2가지가 존재한다. 하나는 Tensor, 다른 하나는 Operation. 각각 모두 실행시킬수 있는데, Tensor는 `eval`이라는 내부 메소드로, Operation은 `run`이라는 내부 메소드로 각자 실행시킬 수 있다.

In [5]:
# Tensor을 실행시킴
print("y의 type : ",type(y))
y.eval(feed_dict={X:[[1.2,2.1,3.2]]},session=sess) # Tensor의 출력값은 늘 존재

y의 type :  <class 'tensorflow.python.framework.ops.Tensor'>


array([[-6.3482347 ,  0.43590015,  2.997642  ]], dtype=float32)

In [6]:
# Operation을 실행시킴
print("init_op의 type : ",type(init_op))
init_op.run(session=sess) # Operation의 출력값은 None

init_op의 type :  <class 'tensorflow.python.framework.ops.Operation'>


그럼 도대체 Operation은 왜 실행시킬까? 제일 중요한 Operation, `train_op`를 생각하면 된다. 이것의 핵심은 주어진 Loss값에 따라, `weight`들을 **갱신**하는데에 있기 때문이다. init_op도 반환값이 없지만, 실제로는 Weight들의 값을 바꾸어 주기 때문에, 위를 실행시킨후 Weight의 값을 보면, 바뀌는 것을 확인할 수 있다.

In [7]:
print("Before Init")
print(W.eval(session=sess))

print("\nRun Init\n")
init_op.run(session=sess)

print("After Init")
print(W.eval(session=sess))

Before Init
[[-0.46453178 -0.7807862  -1.4148815 ]
 [ 1.0920193   0.8149566   1.1573392 ]
 [-0.4756396   0.5213769  -2.8332312 ]]

Run Init

After Init
[[-0.13329014 -1.5023383  -1.2689217 ]
 [-0.47134164 -1.0523499  -1.6041415 ]
 [-0.74693674 -1.3982364   2.3389347 ]]


근데 Graph의 요소를 꼭 `tensor`나 `operation` 객체로 지정해줘야 하나? 객체로 할 필요가 없다, 왜냐하면, Graph의 요소들은 `name`이라는 unique 객체가 존재하기 때문이다. 

#### Operation

In [8]:
sess.run("init") # == sess.run(init_op)

> 위의 코드는 실상 아래와 동일하게 동작한다.

In [9]:
op = graph.get_operation_by_name('init')
sess.run(op)

#### Tensor

In [10]:
sess.run("MatMul:0",feed_dict={X:[[1.2,2.1,3.2]]}) # == sess.run(y, feed_dict={})

array([[ 3.0194035,  3.1660342, -4.3950477]], dtype=float32)

In [11]:
tensor = graph.get_tensor_by_name('MatMul:0')
sess.run(tensor, feed_dict={X:[[1.2,2.1,3.2]]})

array([[ 3.0194035,  3.1660342, -4.3950477]], dtype=float32)

> 꼭 fetches에만 가능한가?  아니다, feed_dict에도 가능하다!

In [12]:
sess.run("MatMul:0",feed_dict={"X:0":[[1.2,2.1,3.2]]})

array([[ 3.0194035,  3.1660342, -4.3950477]], dtype=float32)

### Jupyter 환경을 위해 만들어진 Session, `tf.InteractiveSession`

> 잘 쓰이진 않지만, 이런 Session 타입도 있구나~ 로 알아두면 좋은것.

In [13]:
sess = tf.InteractiveSession(graph=graph)

In [14]:
init_op.run() # 따로 Session을 명시하지 않더라도, InteractiveSession으로 잡음

# Tensor을 실행시킴
print("y의 type : ",type(y))
y.eval(feed_dict={X:[[1.2,2.1,3.2]]}) # Tensor의 출력값은 늘 존재

y의 type :  <class 'tensorflow.python.framework.ops.Tensor'>


array([[ 3.8774784, -4.8154936, -1.5008044]], dtype=float32)

> 딱히 그리 쓸모 있진 않다. 왜냐하면, 보통 `sess.run()`형태로 돌리기 때문이다.

----

### Session을 저장하자, `tf.train.Saver`

Session의 정보는 크게 `Graph`에 대한 정보, 그리고 현재 학습된 weights들, 즉 `Variable`의 값들에 대한 정보로 구성되어 있다. 그래서 `tf.train.Saver`는 `Variable`을 중심으로 작동한다.

아래가 공식문서의 설명이다.

````
reference : https://www.tensorflow.org/api_docs/python/tf/train/Saver

Saves and restores variables.

See Variables for an overview of variables, saving and restoring.

The Saver class adds ops to save and restore variables to and from checkpoints. It also provides convenience methods to run these ops.

Checkpoints are binary files in a proprietary format which map variable names to tensor values. The best way to examine the contents of a checkpoint is to load it using a Saver.
````

tf.train.Saver는 기본적으로 현재 Session에 있는 Variable들의 값을 저장하기 위해 만들어졌다. Variable는 학습함에 따라, 계속 값이 변하게 된다. 그렇기 때문에, 기본적으로 `Checkpoint`라는 타입으로 저장되는데, 이는 step 별로 Variable의 값들을 저장하는 형태로 진행됨을 의미한다.

In [66]:
graph = tf.Graph()

with graph.as_default():
    X = tf.placeholder(tf.float32, (1,3), name='X')    
    W = tf.get_variable("weight", shape=(3,3), 
                        initializer=tf.initializers.random_normal())
    y = tf.matmul(X,W)
    init_op = tf.global_variables_initializer()
    
sess = tf.Session(graph=graph)
sess.run(init_op)

In [67]:
# 저장할 variable list를 가져오기
var_list = graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
# 저장할 variable list들을 지정
saver = tf.train.Saver(var_list)

# sess에 저장된 variable들을 저장
saver.save(sess,"./basic_graph")

'./basic_graph'

#### 이렇게 할 경우 총 4개의 파일로 나누어 저장된다. 

* `basic-graph.data-00000-of-00001` : training variable의 값들
* `basic-graph.index` : graph와 variable의 값을 매칭하기 위한 매칭 table
* `basic-graph.meta` : graph의 정보 <- write_meta_graph=False 하면 저장되지 않음
* `checkpoint` : 가장 최근에 저장된 checkpoint의 이름

----

### 텐서플로우도 Numpy처럼...!, `Eager Execution`

`eager execution`의 목적은 바로 `build time`과 `Run time`의 일치. 텐서플로우의 코딩 방식은 파이썬의 이점인 interpreter의 장점을 살리지 못한다. 마치 numpy로 코드를 짜는 거 처럼, 즉각즉각 어떤 값이 나오는지 디버깅하기가 매우 어렵다. 


> 이것은 차후에 다시 다루도록 하겠다. autograph & gradientTape 등이 섞여 있어, 텐서플로우와 너무 많이 다른 스타일... 되려 복잡성을 증대시킨 느낌이 든다. 심지어 placeholder가 안되면서, 코드의 구조도 많이 바뀌어야 해서 이용해 먹기가 매우 빡셈.

오히려 이런 것 보다, graph에다가 내부 메소드를 추가하는 편이 좀 더 편하지 않을까 싶음.

In [ ]:
def decorate_graph(graph_class):
    def run(self, fetches, feed_dict=None):
        tf.random.set_random_seed(0)
        with self.as_default():
            with tf.Session(graph=self) as sess:
                sess.run([tf.local_variables_initializer(),
                          tf.global_variables_initializer()])
                return sess.run(fetches, feed_dict)
    graph_class.run = run
    return graph_class

In [ ]:
tf.Graph = decorate_graph(tf.Graph)

In [ ]:
# building the graph
graph = tf.Graph()
with graph.as_default():
    X = tf.placeholder(tf.float32,shape=(1,3),name="X")
    M = tf.constant([[1.2],[2.1],[3.2]])
    y = tf.matmul(X,M)

print(graph.run(y,feed_dict={X:[[1.2,2.1,3.2]]}))

<hr>

Copyright(c) 2019 by Public AI. All rights reserved.<br>
Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/01/24
<hr>